# 1장, 서울시 구별 CCTV 현황 분석

## 1-1 CCTV 현황과 인구 현황 데이터 구하기
---

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

## 1-2 파이썬에서 텍스트 파일과 엑셀 파일 읽기 - pandas

### CCTV_in_Seoul.csv 데이터 불러오기

In [3]:
Data_Path = '/content/drive/MyDrive/파이썬으로_데이터_주무르기/1장 서울시 구별 CCTV 현황 분석/data/'

In [4]:
CCTV_Seoul = pd.read_csv(Data_Path + '01. CCTV_in_Seoul.csv', encoding = 'utf-8')
CCTV_Seoul.head()

,기관명,소계,2013년도 이전,2014년,2015년,2016년
0,강남구,2780,1292,430,584,932
1,강동구,773,379,99,155,377
2,강북구,748,369,120,138,204
3,강서구,884,388,258,184,81
4,관악구,1496,846,260,390,613


### Column이름 변경하기

In [5]:
CCTV_Seoul.columns

Index(['기관명', '소계', '2013년도 이전', '2014년', '2015년', '2016년'], dtype='object')

In [6]:
CCTV_Seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   기관명        25 non-null     object
 1   소계         25 non-null     int64 
 2   2013년도 이전  25 non-null     int64 
 3   2014년      25 non-null     int64 
 4   2015년      25 non-null     int64 
 5   2016년      25 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 1.3+ KB


In [7]:
CCTV_Seoul.columns[0]

'기관명'

In [8]:
CCTV_Seoul.rename(columns = {CCTV_Seoul.columns[0] : '구별'}, inplace = True) # inplace = True 는, 실제 CCTV_Seoul이라는 "변수의 내용을 갱신"
CCTV_Seoul.head()

,구별,소계,2013년도 이전,2014년,2015년,2016년
0,강남구,2780,1292,430,584,932
1,강동구,773,379,99,155,377
2,강북구,748,369,120,138,204
3,강서구,884,388,258,184,81
4,관악구,1496,846,260,390,613


### population_in_Seoul.xls 데이터 불러오기

In [9]:
# pop_Seoul = pd.read_excel(Data_Path + '01. population_in_Seoul.xls')
# pop_Seoul.head()

,기간,자치구,세대,인구,인구.1,인구.2,인구.3,인구.4,인구.5,인구.6,인구.7,인구.8,세대당인구,65세이상고령자
0,기간,자치구,세대,합계,합계,합계,한국인,한국인,한국인,등록외국인,등록외국인,등록외국인,세대당인구,65세이상고령자
1,기간,자치구,세대,계,남자,여자,계,남자,여자,계,남자,여자,세대당인구,65세이상고령자
2,2017.1/4,합계,4202888,10197604,5000005,5197599,9926968,4871560,5055408,270636,128445,142191,2.36,1321458
3,2017.1/4,종로구,72654,162820,79675,83145,153589,75611,77978,9231,4064,5167,2.11,25425
4,2017.1/4,중구,59481,133240,65790,67450,124312,61656,62656,8928,4134,4794,2.09,20764


- 오류를 검색해 보았더니, pandas openpyxl의 excel 관련 버전 충돌이라고 한다
- 위의 문제로 해결이 안됐고, 이번에는 xlrd 버전 관련 에러가 나서 업그레이드 후 '다시시작 및 모두실행'을 하니 문제가 해결 되었다.

여전히 의문1: 왜 이쪽은 encoding 관련 오류가 날까?

In [10]:
!pip install --upgrade xlrd #완료 후에 다시시작 및 모두실행을 하였다.

In [11]:
pop_Seoul = pd.read_excel(Data_Path + '01. population_in_Seoul.xls')
pop_Seoul.head()

,기간,자치구,세대,인구,인구.1,인구.2,인구.3,인구.4,인구.5,인구.6,인구.7,인구.8,세대당인구,65세이상고령자
0,기간,자치구,세대,합계,합계,합계,한국인,한국인,한국인,등록외국인,등록외국인,등록외국인,세대당인구,65세이상고령자
1,기간,자치구,세대,계,남자,여자,계,남자,여자,계,남자,여자,세대당인구,65세이상고령자
2,2017.1/4,합계,4202888,10197604,5000005,5197599,9926968,4871560,5055408,270636,128445,142191,2.36,1321458
3,2017.1/4,종로구,72654,162820,79675,83145,153589,75611,77978,9231,4064,5167,2.11,25425
4,2017.1/4,중구,59481,133240,65790,67450,124312,61656,62656,8928,4134,4794,2.09,20764


엑셀 파일 자체의 head()가 저렇게 쓰여있기에, 이상하게 보인다.
때문에 ...

### 특정 '줄' 부터 읽고 // 특정 '열'만 읽기


In [12]:
pop_Seoul = pd.read_excel(Data_Path + '01. population_in_Seoul.xls',
                          header = 2,
                          usecols = 'B, D, G, J, N',)

pop_Seoul.head()

,자치구,계,계.1,계.2,65세이상고령자
0,합계,10197604.0,9926968.0,270636.0,1321458.0
1,종로구,162820.0,153589.0,9231.0,25425.0
2,중구,133240.0,124312.0,8928.0,20764.0
3,용산구,244203.0,229456.0,14747.0,36231.0
4,성동구,311244.0,303380.0,7864.0,39997.0


### 이름을 새로 지정하기

In [16]:
pop_Seoul.rename(columns = {pop_Seoul.columns[0] : '구별',
                            pop_Seoul.columns[1] : '인구수',
                            pop_Seoul.columns[2] : '한국인',
                            pop_Seoul.columns[3] : '외국인',
                            pop_Seoul.columns[4] : '고령자'}, inplace = True)

pop_Seoul.head()

,구별,인구수,한국인,외국인,고령자
0,합계,10197604.0,9926968.0,270636.0,1321458.0
1,종로구,162820.0,153589.0,9231.0,25425.0
2,중구,133240.0,124312.0,8928.0,20764.0
3,용산구,244203.0,229456.0,14747.0,36231.0
4,성동구,311244.0,303380.0,7864.0,39997.0


## 1-3 pandas 기초 익히기

In [17]:
import pandas as pd
import numpy as np

### Serise 만들기.
- 대괄호로 list를 생성하였습니다.
- `np.nan`을 넣어 NaN(Not a Number)도 넣었습니다. 

In [18]:
s = pd.Series([1,3,5,np.nan,6,8])
s

0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64

pandas에는 날짜형 데이터인 date_range가 있습니다.
- 기본 날짜를 지정하고 periods옵션으로 6일간으로 지정 하였습니다.

In [19]:
dates = pd.date_range('20130101', periods = 6)
dates

DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06'],
              dtype='datetime64[ns]', freq='D')

DataFrame 유형의 데이터를 만듭니다.
- 6행 4열의 random변수를 만들었습니다.
- index에는 위에서 만든 날짜형 데이터인 dates를 넣었습니다.
- columns 에는 ['A', 'B', 'C', 'D'] 를 넣어주었습니다.

In [20]:
df = pd.DataFrame(np.random.randn(6,4), index = dates, columns = ['A', 'B', 'C', 'D'])
df

,A,B,C,D
2013-01-01,-0.491532,-1.196635,-0.315436,1.762794
2013-01-02,-1.730919,-2.017910,2.133938,0.699490
2013-01-03,0.859902,-0.116425,-1.655446,0.117389
2013-01-04,-1.199065,-1.758823,-0.667774,0.949137
2013-01-05,-0.262343,-0.264856,0.754198,0.084439
2013-01-06,0.560102,1.096006,1.295520,-0.181213


`head()` 라고하면 첫 5행만 보여주고, 괄호안에 숫자를 넣으면 그 만큼의 행을 볼 수 있습니다.

In [21]:
df.head(3)

,A,B,C,D
2013-01-01,-0.491532,-1.196635,-0.315436,1.762794
2013-01-02,-1.730919,-2.017910,2.133938,0.699490
2013-01-03,0.859902,-0.116425,-1.655446,0.117389


`index`, `columns` 명령을 이용하면, pandas의 DataFrame의 컬럼과 인덱스를 확인할 수 있습니다.

In [22]:
df.index

DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06'],
              dtype='datetime64[ns]', freq='D')

In [23]:
df.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

그리고 그 안에 들어가는 내용은 `value` 명령으로 확인할 수 있습니다. 

In [25]:
df.values

array([[-0.49153175, -1.19663519, -0.31543553,  1.76279389],
       [-1.73091864, -2.01790995,  2.13393759,  0.69948971],
       [ 0.8599025 , -0.1164246 , -1.65544588,  0.11738926],
       [-1.19906503, -1.75882277, -0.66777376,  0.94913658],
       [-0.2623426 , -0.26485581,  0.7541979 ,  0.08443864],
       [ 0.5601022 ,  1.09600632,  1.29552039, -0.18121344]])

또한, `.info()`명령을 사용하면 DataFrame의 개요를 알 수 있습니다.

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6 entries, 2013-01-01 to 2013-01-06
Freq: D
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A       6 non-null      float64
 1   B       6 non-null      float64
 2   C       6 non-null      float64
 3   D       6 non-null      float64
dtypes: float64(4)
memory usage: 240.0 bytes


`describe()` 명령을 사용하면 통계적 개요를 확인할 수 있습니다.
- count와 평균, min, max, 1/4 지점, 표준편차까지 한 번에 알 수 있습니다.
- 이 때, values가 숫자가 아니라 문자라고 하더라도, 그에 맞는 개요가 나타납니다.

In [27]:
df.describe()

,A,B,C,D
count,6.000000,6.000000,6.000000,6.000000
mean,-0.377309,-0.709774,0.257500,0.572006
std,0.994308,1.171291,1.392046,0.719471
min,-1.730919,-2.017910,-1.655446,-0.181213
25%,-1.022182,-1.618276,-0.579689,0.092676
50%,-0.376937,-0.730745,0.219381,0.408439
75%,0.354491,-0.153532,1.160190,0.886725
max,0.859902,1.096006,2.133938,1.762794


`sort_values` 명령은, `by`로 지정된 컬럼을 기준으로 정렬합니다. 

`ascending`옵션을 사용하면, 내림차순이나 오름차순으로 정렬시킬 수 있습니다.
- (False = 내림차순, True = 오름차순)

In [28]:
df.sort_values(by='B', ascending = False)

,A,B,C,D
2013-01-06,0.560102,1.096006,1.295520,-0.181213
2013-01-03,0.859902,-0.116425,-1.655446,0.117389
2013-01-05,-0.262343,-0.264856,0.754198,0.084439
2013-01-01,-0.491532,-1.196635,-0.315436,1.762794
2013-01-04,-1.199065,-1.758823,-0.667774,0.949137
2013-01-02,-1.730919,-2.017910,2.133938,0.699490


`변수명`을 적고, 실행을 하면 그 내용이 나타납니다.
- 위에서 sort를 저장한것이 아니기 때문에, 다시 원래의 표가 나옵니다.
- 내용이 너무 많은 경우에는 Jupyter Notebook에서 내용을 줄여서 보여줍니다.

In [30]:
df

,A,B,C,D
2013-01-01,-0.491532,-1.196635,-0.315436,1.762794
2013-01-02,-1.730919,-2.017910,2.133938,0.699490
2013-01-03,0.859902,-0.116425,-1.655446,0.117389
2013-01-04,-1.199065,-1.758823,-0.667774,0.949137
2013-01-05,-0.262343,-0.264856,0.754198,0.084439
2013-01-06,0.560102,1.096006,1.295520,-0.181213


DataFrame으로 선언 된 변수의 경우 변수면 뒤에 `[columns명]`을 붙여주면 해당 칼럼만 Series로 보여줍니다.

In [31]:
df['A']

2013-01-01   -0.491532
2013-01-02   -1.730919
2013-01-03    0.859902
2013-01-04   -1.199065
2013-01-05   -0.262343
2013-01-06    0.560102
Freq: D, Name: A, dtype: float64

중간부터 보고싶다면, 행의 범위만 지정해서 볼 수 있습니다

In [33]:
df[2:6]

,A,B,C,D
2013-01-03,0.859902,-0.116425,-1.655446,0.117389
2013-01-04,-1.199065,-1.758823,-0.667774,0.949137
2013-01-05,-0.262343,-0.264856,0.754198,0.084439
2013-01-06,0.560102,1.096006,1.295520,-0.181213


특정 행만 지정해서 볼 수도 있습니다.

In [34]:
df['20130102':'20130104']

,A,B,C,D
2013-01-02,-1.730919,-2.017910,2.133938,0.699490
2013-01-03,0.859902,-0.116425,-1.655446,0.117389
2013-01-04,-1.199065,-1.758823,-0.667774,0.949137


위에서 만든 dates 변수를 이용하여 특정 날짜의 데이터만 보고싶으면, `df.loc`명령을 사용합니다.
- loc는 location 옵션으로, 슬라이싱 할 때 loc옵션을 이용하여 위치 값을 지정할 수 있습니다.

In [40]:
print(dates[0],'\n','='*10,'\n')
df.loc[dates[0]] #dates[0] 의 행만 보여줍니다

2013-01-01 00:00:00 



A   -0.491532
B   -1.196635
C   -0.315436
D    1.762794
Name: 2013-01-01 00:00:00, dtype: float64